In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [19]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.race_lap_time`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_4551/4151465537.py:10: FutureWarning:

read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq



In [220]:
df_bq=df_bq2[(df_bq2["GP"]=='Austrian Grand Prix')&(df_bq2["Year"]==2024)]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_4551/248577147.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
2753,0 days 00:56:51.843000,ALB,23,0 days 00:01:20.507000,1,1.0,NaT,NaT,NaT,0 days 00:00:33.816000,0 days 00:00:23.126000,NaT,0 days 00:56:28.957000,0 days 00:56:51.882000,322.0,221.0,271.0,275.0,False,MEDIUM,1.0,True,Williams,0 days 00:55:31.047000,NaT,1,15.0,False,,False,False,2024,Austrian Grand Prix
2869,0 days 00:58:04.171000,ALB,23,0 days 00:01:12.328000,2,1.0,NaT,NaT,0 days 00:00:17.758000,0 days 00:00:32.451000,0 days 00:00:22.119000,0 days 00:57:09.579000,0 days 00:57:42.030000,0 days 00:58:04.149000,327.0,222.0,279.0,313.0,True,MEDIUM,2.0,True,Williams,0 days 00:56:51.843000,NaT,1,15.0,False,,False,True,2024,Austrian Grand Prix
147444,0 days 00:59:15.660000,ALB,23,0 days 00:01:11.489000,3,1.0,NaT,NaT,0 days 00:00:17.397000,0 days 00:00:32.103000,0 days 00:00:21.989000,0 days 00:58:21.546000,0 days 00:58:53.649000,0 days 00:59:15.638000,320.0,223.0,278.0,308.0,True,MEDIUM,3.0,True,Williams,0 days 00:58:04.171000,NaT,1,15.0,False,,False,True,2024,Austrian Grand Prix
147420,0 days 01:00:27.166000,ALB,23,0 days 00:01:11.506000,4,1.0,NaT,NaT,0 days 00:00:17.654000,0 days 00:00:32.011000,0 days 00:00:21.841000,0 days 00:59:33.292000,0 days 01:00:05.303000,0 days 01:00:27.144000,310.0,225.0,278.0,302.0,False,MEDIUM,4.0,True,Williams,0 days 00:59:15.660000,NaT,1,15.0,False,,False,True,2024,Austrian Grand Prix
147379,0 days 01:01:38.568000,ALB,23,0 days 00:01:11.402000,5,1.0,NaT,NaT,0 days 00:00:17.404000,0 days 00:00:32.080000,0 days 00:00:21.918000,0 days 01:00:44.548000,0 days 01:01:16.628000,0 days 01:01:38.546000,315.0,224.0,278.0,302.0,True,MEDIUM,5.0,True,Williams,0 days 01:00:27.166000,NaT,1,15.0,False,,False,True,2024,Austrian Grand Prix


In [221]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [222]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [223]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)


In [224]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_Sectors=GP_Sectors.fillna(0)
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
2753,ALB,1,0.0,33.816,23.126,00:01:20.507000,80.507,MEDIUM,2024,Austrian Grand Prix,15.0,0.0,18.0,14.0
8768,ALO,1,0.0,33.491,23.130,00:01:19.908000,79.908,MEDIUM,2024,Austrian Grand Prix,13.0,0.0,14.0,15.0
305747,BOT,1,0.0,33.727,23.487,00:01:20.918000,80.918,MEDIUM,2024,Austrian Grand Prix,16.0,0.0,17.0,18.0
34128,GAS,1,0.0,33.129,22.579,00:01:19.109000,79.109,MEDIUM,2024,Austrian Grand Prix,11.0,0.0,10.0,11.0
333158,HAM,1,0.0,32.193,21.528,00:01:14.840000,74.840,MEDIUM,2024,Austrian Grand Prix,4.0,0.0,3.0,4.0


In [225]:
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [226]:
#get a combined view for fastest drivers in the entire lap, S1, S2, and S3
Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
Fast_Laps=Fast_Laps.reset_index(drop=True)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound
0,Austrian Grand Prix,2024,2,00:01:09.813000,69.813,NOR,MEDIUM,17.330,HAM,MEDIUM,31.155,HUL,MEDIUM,21.011,NOR,MEDIUM
1,Austrian Grand Prix,2024,3,00:01:09.857000,69.857,VER,MEDIUM,17.235,PER,MEDIUM,31.065,VER,MEDIUM,21.157,RUS,MEDIUM
2,Austrian Grand Prix,2024,4,00:01:09.903000,69.903,VER,MEDIUM,17.274,SAI,MEDIUM,31.180,VER,MEDIUM,21.152,VER,MEDIUM
3,Austrian Grand Prix,2024,5,00:01:10.005000,70.005,VER,MEDIUM,17.320,HAM,MEDIUM,31.153,VER,MEDIUM,21.178,VER,MEDIUM
4,Austrian Grand Prix,2024,6,00:01:10.238000,70.238,VER,MEDIUM,17.401,RIC,MEDIUM,31.295,VER,MEDIUM,21.221,NOR,MEDIUM


In [227]:
Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,Austrian Grand Prix,2024,2,00:01:09.813000,69.813,NOR,MEDIUM,17.330,HAM,MEDIUM,31.155,HUL,MEDIUM,21.011,NOR,MEDIUM,#FF8000
1,Austrian Grand Prix,2024,3,00:01:09.857000,69.857,VER,MEDIUM,17.235,PER,MEDIUM,31.065,VER,MEDIUM,21.157,RUS,MEDIUM,#3860C6
2,Austrian Grand Prix,2024,4,00:01:09.903000,69.903,VER,MEDIUM,17.274,SAI,MEDIUM,31.180,VER,MEDIUM,21.152,VER,MEDIUM,#3860C6
3,Austrian Grand Prix,2024,5,00:01:10.005000,70.005,VER,MEDIUM,17.320,HAM,MEDIUM,31.153,VER,MEDIUM,21.178,VER,MEDIUM,#3860C6
4,Austrian Grand Prix,2024,6,00:01:10.238000,70.238,VER,MEDIUM,17.401,RIC,MEDIUM,31.295,VER,MEDIUM,21.221,NOR,MEDIUM,#3860C6


In [228]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig = go.Figure(go.Table
 (header={"values": Fast_Laps.columns},
  cells=dict(
    values=Fast_Laps.T.values,
    fill_color=[Fast_Laps.color],
    align='center', font=dict(color='white', size=14)
    ))
 )

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": Fast_Laps.T.values
                                if c == "All"
                                else Fast_Laps.loc[Fast_Laps["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + Fast_Laps["LapNumber"].unique().tolist()
            ]
        }
    ]
)
fig.update_layout(
    title="Fastest Lap and Sector Times For Ordered By Lap Number",
        title_x=0.5,
        # template="plotly_dark",

    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    height=2500,
    title_font_family="PT Sans Narrow"
)

fig.show()

In [229]:
fig.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Number.html".format(year,gp))

In [230]:
LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
LapTime_ordered.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
69,Austrian Grand Prix,2024,70,00:01:07.694000,67.694,ALO,SOFT,17.109,PER,MEDIUM,30.127,ALO,SOFT,20.346,VER,SOFT,#229971
67,Austrian Grand Prix,2024,68,00:01:07.719000,67.719,VER,SOFT,17.142,VER,SOFT,30.279,VER,SOFT,20.298,VER,SOFT,#3860C6
52,Austrian Grand Prix,2024,53,00:01:08.016000,68.016,NOR,MEDIUM,16.897,NOR,MEDIUM,30.558,NOR,MEDIUM,20.561,NOR,MEDIUM,#FF8000
68,Austrian Grand Prix,2024,69,00:01:08.321000,68.321,VER,SOFT,17.080,PER,MEDIUM,30.674,VER,SOFT,20.395,VER,SOFT,#3860C6
53,Austrian Grand Prix,2024,54,00:01:08.429000,68.429,NOR,MEDIUM,16.996,NOR,MEDIUM,30.890,NOR,MEDIUM,20.543,NOR,MEDIUM,#FF8000


In [231]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig2 = go.Figure(go.Table
 (header={"values": LapTime_ordered.columns},
  cells=dict(
    values=LapTime_ordered.T.values,
    fill_color=[LapTime_ordered.color],
    align='center', font=dict(color='white', size=15)
    ))
 )

fig2.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": LapTime_ordered.T.values
                                if c == "All"
                                else LapTime_ordered.loc[LapTime_ordered["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + LapTime_ordered["LapNumber"].unique().tolist()
            ]
        }
    ]
)
fig2.update_layout(
    title="Fastest Lap and Sector Times Ordered By Lap Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=16,
        color="Black"
    ),
    height=2500,
    title_font_family="PT Sans Narrow"
)

fig2.show()

In [232]:
fig2.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year,gp))

In [233]:
#get a shortened view for fastest drivers in the entire lap, S1, S2, and S3

Fast_Laps_short=Fast_Laps[["GP","Year","LapNumber","Fastest Lap","Fastest S1","Fastest S2","Fastest S3","color"]].sort_values(by=["LapNumber"])
Fast_Laps_short=Fast_Laps_short.reset_index(drop=True)
Fast_Laps_short.head()

,GP,Year,LapNumber,Fastest Lap,Fastest S1,Fastest S2,Fastest S3,color
0,Austrian Grand Prix,2024,2,NOR,HAM,HUL,NOR,#FF8000
1,Austrian Grand Prix,2024,3,VER,PER,VER,RUS,#3860C6
2,Austrian Grand Prix,2024,4,VER,SAI,VER,VER,#3860C6
3,Austrian Grand Prix,2024,5,VER,HAM,VER,VER,#3860C6
4,Austrian Grand Prix,2024,6,VER,RIC,VER,NOR,#3860C6


In [234]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig = go.Figure(go.Table
 (header={"values": Fast_Laps_short.columns},
  cells=dict(
    values=Fast_Laps_short.T.values,
    height=50,
    fill_color=[Fast_Laps_short.color],
    align='center', font=dict(color='white', size=20)
    ))
 )

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": Fast_Laps_short.T.values
                                if c == "All"
                                else Fast_Laps_short.loc[Fast_Laps_short["LapNumber"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + Fast_Laps_short["LapNumber"].unique().tolist()
            ]
        }
    ]
)
fig.update_layout(
    title="Fastest Lap and Sector Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=20,
        color="Black"
    ),
    height=2500,
    title_font_family="PT Sans Narrow"
)

fig.show()

In [235]:
#get a view for fastest lap in the entire race
Fastest_Lap_Race=GP_Rk[GP_Rk["Race_RK"]==1][["Year","GP","Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound"]].reset_index(drop=True)
Fastest_Lap_Race

,Year,GP,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound
0,2024,Austrian Grand Prix,ALO,70,17.196,30.127,20.371,00:01:07.694000,67.694,SOFT


In [219]:
#Multiple plotly filters code
import plotly.graph_objects as go

df = pd.DataFrame(
    {
        "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
        "A_item": [2, 8, 0],
        "B_item": [1, 7, 10],
        "C_item": [9, 2, 9],
        "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
    }
)

fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
fig.update_layout(
    updatemenus=[
        {
            "y": 1 - (i / 5),
            "buttons": [
                {
                    "label": c,
                    "method": "restyle",
                    "args": [
                        {
                            "cells": {
                                "values": df.T.values
                                if c == "All"
                                else df.loc[df[menu].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + df[menu].unique().tolist()
            ],
        }
        for i, menu in enumerate(["Channel_type", "Date"])
    ]
)

In [202]:
#Single Plotly filter code
import plotly.graph_objects as go

df = pd.DataFrame(
    {
        "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
        "A_item": [2, 8, 0],
        "B_item": [1, 7, 10],
        "C_item": [9, 2, 9],
        "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
    }
)

fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": c,
                    "method": "update",
                    "args": [
                        {
                            "cells": {
                                "values": df.T.values
                                if c == "All"
                                else df.loc[df["Channel_type"].eq(c)].T.values
                            }
                        }
                    ],
                }
                for c in ["All"] + df["Channel_type"].unique().tolist()
            ]
        }
    ]
)